In [1]:
import neo4j

host = "localhost"
username = "neo4j"
password = "1234qwer"


driver = neo4j.GraphDatabase.driver(f'bolt://localhost:7687', auth=(username, password))
session = driver.session()

def create_graph(query):
    session.run(query)

In [2]:
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [3]:
sentence = "Customer is able to check inventory stock"
sentence_embedding = get_embedding(sentence)

In [4]:
records, summary, key  = driver.execute_query("""
    MATCH (s:Requirement)
    WITH s, vector.similarity.cosine(s.sentence_embedding, $embedding) as score
    RETURN s.id as id, s.app as app, s.sentence as sentence, score ORDER BY score DESC LIMIT 5""", embedding=sentence_embedding)

result = []
for record in records:
    result.append({"app": record['app'], "sentence": record['sentence'], "score": record['score']})

In [5]:
import pandas as pd
df_result = pd.DataFrame(result)

In [6]:
df_result[:5]

,app,sentence,score
0,POS Connect,A customer can inspect items online add them t...,0.950500
1,Odoo Purchase,"Keep track of stock, incoming orders and produ...",0.946841
2,POS Connect,As items are sold the online inventory is upda...,0.940191
3,SCATS,The system must allow the Sales Person to sear...,0.940115
4,SMS,The system shall display the details of the ex...,0.937592
